## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-09-12-01-00 +0000,missionlocal,San Francisco teachers strike: Here’s where bo...,https://missionlocal.org/2026/02/sfusd-teacher...
1,2026-02-09-12-01-00 +0000,wsj,Both parties face enormous challenges and inte...,https://www.wsj.com/politics/policy/six-politi...
2,2026-02-09-11-58-29 +0000,nyt,Jimmy Lai’s 20-Year Sentence Follows Beijing’s...,https://www.nytimes.com/2026/02/09/world/asia/...
3,2026-02-09-11-47-00 +0000,wsj,European Gas Prices Fall Despite Low Storage L...,https://www.wsj.com/articles/european-gas-pric...
4,2026-02-09-11-46-38 +0000,bbc,Barry M bought by make-up for men brand Warpaint,https://www.bbc.com/news/articles/c98q50gd2vmo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2437/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
223,super,25
224,bowl,24
55,trump,16
221,bad,15
366,election,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
116,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...,125
27,2026-02-09-10-55-52 +0000,nyt,The Wedding During Bad Bunny’s Super Bowl Half...,https://www.nytimes.com/2026/02/09/us/super-bo...,114
112,2026-02-09-03-08-19 +0000,nyt,"Lady Gaga, Ricky Martin and Cardi B Join Bad B...",https://www.nytimes.com/2026/02/08/us/bad-bunn...,113
51,2026-02-09-09-36-38 +0000,bbc,Bad Bunny makes history as Trump criticises 't...,https://www.bbc.com/news/articles/c394g7nnzmzo...,112
101,2026-02-09-04-00-40 +0000,latimes,"Bad Bunny fit in an actual wedding, with cake,...",https://www.latimes.com/california/story/2026-...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
116,125,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...
52,65,2026-02-09-09-11-41 +0000,nypost,China critic and former media tycoon Jimmy Lai...,https://nypost.com/2026/02/09/world-news/china...
47,58,2026-02-09-09-47-25 +0000,nyt,Japan Stocks Surge on Takaichi’s Landslide Ele...,https://www.nytimes.com/2026/02/08/business/ja...
213,44,2026-02-08-18-04-45 +0000,nypost,Iran sentences locked-up Nobel Peace Prize win...,https://nypost.com/2026/02/08/world-news/iran-...
202,36,2026-02-08-19-17-15 +0000,nypost,London’s Camden Goods Yard poised to draw New ...,https://nypost.com/2026/02/08/business/londons...
217,35,2026-02-08-17-35-40 +0000,nypost,UK PM Keir Starmer’s top aide Morgan McSweeney...,https://nypost.com/2026/02/08/world-news/uk-pm...
72,31,2026-02-09-06-45-50 +0000,nypost,"Juan Pablo Guanipa, ally of Venezuelan opposit...",https://nypost.com/2026/02/09/world-news/venez...
162,30,2026-02-08-23-01-00 +0000,startribune,Lindsey Vonn breaks her left leg in Olympic do...,https://www.startribune.com/2026-winter-olympi...
138,29,2026-02-09-00-57-00 +0000,nypost,Japanese PM Sanae Takaichi’s party sweeps to m...,https://nypost.com/2026/02/08/world-news/japan...
58,28,2026-02-09-08-35-25 +0000,nypost,Washington mom sentenced after killing 8-year-...,https://nypost.com/2026/02/09/us-news/washingt...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
